## IMPORT

In [1]:
import twitter
import spacy
import nltk
from spacy import displacy
import markovify
import gzip
import re
import json
import spacy

In [2]:
nlp = spacy.load('en_core_web_lg')

## RETRIEVE TWEETS: API

In [5]:
twitter_credentials = {
    'consumer_key':        'wbz78wFd0ywcShiTvqgDUV2ry',
    'consumer_secret':     '2qj0P3fygqa0n2LqU6M8LV485OWIAvXWEQOEVLWFNUBdKDcgjz',
    'access_token_key':    '80578720-t7bH4zwD6Q6sUQEFeCb8211wH04Y9ul0EWECo2ofU',
    'access_token_secret': 'PSsX8R4agpxAII9XCYHqE74KObPRWfl9tdG4Xd07olOn6'
}

In [6]:
twapi = twitter.Api(consumer_key=twitter_credentials['consumer_key'],
                    consumer_secret=twitter_credentials['consumer_secret'],
                    access_token_key=twitter_credentials['access_token_key'],
                    access_token_secret=twitter_credentials['access_token_secret'],
                    tweet_mode='extended')

In [17]:
# screen_name = 'nigel_farage'
# screen_name = 'realdonaldtrump'
screen_name = 'borisjohnson'

In [18]:
#  Retrieve as much tweet history as possible for training purposes
timeline = []
max_id = None
while True:
    print('max_id for iteration: {}'.format(max_id))
    tweets = twapi.GetUserTimeline(screen_name=screen_name, include_rts=False, count=200, max_id=max_id)
    timeline += tweets
    
    next_max_id = min(tweets, key=lambda t: t.id).id
    if next_max_id == max_id:
        break
    max_id = next_max_id

max_id for iteration: None
max_id for iteration: 1156655029803192320
max_id for iteration: 1123597392564105216
max_id for iteration: 1045428718682992641
max_id for iteration: 986649042858934273
max_id for iteration: 953970924201619457
max_id for iteration: 908078771345002496
max_id for iteration: 869296982573162497
max_id for iteration: 819497592472174592
max_id for iteration: 774286227558334464
max_id for iteration: 661145791189417984
max_id for iteration: 583374543902941184


## RETRIEVE TWEETS: FILE

In [3]:
tweetfile = '/Users/liangjh/Workspace/corpii/tweets-realdonaldtrump.json'
screen_name = 'realdonaldtrump'

In [4]:
with open(tweetfile) as json_file:
    timeline = json.load(json_file)

In [5]:
timeline[0]

{'source': 'Twitter for iPhone',
 'text': 'Will be looking into the Scott Hapgood case, and the Island of Anguilla. Something looks and sounds very wrong. I know Anguilla will want to see this case be properly and justly resolved! @foxandfriends @SteveDucey  @ainsleyearhardt',
 'created_at': 'Mon Oct 14 14:00:19 +0000 2019',
 'retweet_count': 2485,
 'favorite_count': 8647,
 'is_retweet': False,
 'id_str': '1183744344794320896'}

## PROCESS TWEETS

In [6]:
timeline.reverse()
# Timeline retrieved from twitter api
# timeline_abridged = [{k:v for k,v in elem.AsDict().items() if k in ['created_at', 'full_text', 'id']} 
#                       for elem in timeline]

# Timeline retrieved from archive
timeline_abridged = [{k:v for k,v in elem.items() if k in ['created_at', 'text', 'id']} 
                      for elem in timeline]




In [7]:
#  Save abridged 
# with open('/Users/liangjh/workspace/corpii/tweets-{}.json'.format(screen_name), 'w') as f:
#     json.dump({'data': timeline_abridged}, f)

In [8]:
timeline_text_sanitized = [re.sub(r'(http|https)?:\/\/.*[\r\n]*', '', tline['text']) # or full_text
                           for tline in timeline_abridged]

In [9]:
timeline_merged_corpus = ' '.join(timeline_text_sanitized)

In [9]:
# with open('/Users/liangjh/workspace/corpii/tweets-trump.txt') as f:
#     tweets_raw_curr = f.read()

In [ ]:
#  Parsed document
# doc = nlp(timeline_merged_corpus)

In [73]:
# tweets_combined = ' '.join(tweets_raw)
# doc = nlp(tweets_combined)
# len(tweets_raw)
# len([token for token in doc])

In [10]:
class POSifiedText(markovify.Text):
    def word_split(self, sentence):
        return ['::'.join((word.orth_, word.pos_)) for word in nlp(sentence)]
    def word_join(self, words):
        sentence = ' '.join(word.split('::')[0] for word in words)
        return sentence

In [11]:
tweet_model = POSifiedText(timeline_merged_corpus, state_size=2)

In [60]:
# for i in range(20):
#     print(tweet_model.make_short_sentence(max_chars=150))

In [12]:
with gzip.GzipFile('/Users/liangjh/workspace/corpii/trumpspire-model-02.gz'.format(screen_name), 'w') as fo:
    fo.write(json.dumps(tweet_model.to_json()).encode('utf-8'))